# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [32]:
data_df=pd.read_csv("result/Weather result.csv")
data_df.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Nome,90,US,1578793155,85,64.50,-165.41,17.006,11.30
1,Olafsvik,67,IS,1578793155,63,64.89,-23.71,28.670,6.75
2,Beringovskiy,100,RU,1578793155,86,63.05,179.32,31.496,11.48
3,Port Keats,100,AU,1578793155,74,-14.25,129.55,83.174,1.43
4,San Patricio,40,MX,1578793155,74,19.22,-104.70,82.400,1.05


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [19]:
gmaps.configure(api_key=g_key)
locations=data_df[["Lat","Lng"]]
humidity=data_df["Humidity"]
fig=gmaps.figure()
heat_layer=gmaps.heatmap_layer(locations,weights=humidity,dissipating=False,max_intensity=200,point_radius=5)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [35]:
new_data_df=data_df.loc[data_df["Max Temp"]>70]
new_data_df=new_data_df.loc[new_data_df["Max Temp"]<80]
new_data_df=new_data_df.loc[new_data_df["Wind Speed"]<10]
new_data_df=new_data_df.loc[new_data_df["Cloudiness"]==0]
new_data_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
7,Esperance,0,AU,1578793156,39,-33.87,121.90,78.800,8.70
68,Bethanien,0,NaN,1578793171,34,-26.50,17.16,70.124,2.37
301,Broken Hill,0,AU,1578793226,28,-31.95,141.43,73.400,7.70
351,Malpe,0,IN,1578793239,73,13.35,74.72,71.600,1.50
356,Takoradze,0,GH,1578793240,92,4.88,-1.76,75.506,0.97
429,Calabozo,0,VE,1578793257,61,8.92,-67.43,76.208,3.34
442,Ampanihy,0,MG,1578793260,58,-24.70,44.75,76.046,2.21
483,Sechura,0,PE,1578793270,84,-5.56,-80.82,70.880,4.03
502,Skeldon,0,GY,1578793274,77,5.88,-57.13,78.944,6.67


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [36]:
hotel_df=new_data_df

In [38]:
hotel_df["Hotel Name"]=""
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,Esperance,0,AU,1578793156,39,-33.87,121.90,78.800,8.70,
68,Bethanien,0,NaN,1578793171,34,-26.50,17.16,70.124,2.37,
301,Broken Hill,0,AU,1578793226,28,-31.95,141.43,73.400,7.70,
351,Malpe,0,IN,1578793239,73,13.35,74.72,71.600,1.50,
356,Takoradze,0,GH,1578793240,92,4.88,-1.76,75.506,0.97,
429,Calabozo,0,VE,1578793257,61,8.92,-67.43,76.208,3.34,
442,Ampanihy,0,MG,1578793260,58,-24.70,44.75,76.046,2.21,
483,Sechura,0,PE,1578793270,84,-5.56,-80.82,70.880,4.03,
502,Skeldon,0,GY,1578793274,77,5.88,-57.13,78.944,6.67,


In [61]:
params = {
    "location":"-33.87,121.90",
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

query_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for index,row in hotel_df.iterrows():
    lat=row['Lat']
    lng=row['Lng']
    params['location']=f"{lat},{lng}"
    response = requests.get(query_url, params=params).json()
    try:
        hotel_df.loc[index,"Hotel Name"]=response["results"][0]["name"]
    except:
        pass
hotel_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed,Hotel Name
7,Esperance,0,AU,1578793156,39,-33.87,121.90,78.800,8.70,Hospitality Esperance
68,Bethanien,0,NaN,1578793171,34,-26.50,17.16,70.124,2.37,Bethanie Guesthouse
301,Broken Hill,0,AU,1578793226,28,-31.95,141.43,73.400,7.70,Daydream Motel & Cottages
351,Malpe,0,IN,1578793239,73,13.35,74.72,71.600,1.50,Hotel Sri Krishna Residency
356,Takoradze,0,GH,1578793240,92,4.88,-1.76,75.506,0.97,Raybow International Hotel
429,Calabozo,0,VE,1578793257,61,8.92,-67.43,76.208,3.34,Hotel Giardini
442,Ampanihy,0,MG,1578793260,58,-24.70,44.75,76.046,2.21,Angora Hotel
483,Sechura,0,PE,1578793270,84,-5.56,-80.82,70.880,4.03,Hospedaje Costa Bella
502,Skeldon,0,GY,1578793274,77,5.88,-57.13,78.944,6.67,Rits Hotel


In [62]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in new_data_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [65]:
# Add marker layer ontop of heat map
markers=gmaps.marker_layer(locations,info_box_content=hotel_info)
fig.add_layer(markers)
fig

# Display Map

Figure(layout=FigureLayout(height='420px'))